In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Master Expense Register.csv')

In [3]:
df.head(10)

,3/2/2020,Beginning balance as of 03/02/2020,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,3/2/2020,Counter Credit,"8,618.52","$ 8,618.52",NaN,NaN
1,3/2/2020,"Zelle Transfer Conf# 2de792970; HINKLE, PRESTON",200.00,"$ 8,818.52",Dues,Preston Hinkle
2,3/9/2020,Zelle Transfer Conf# 092BE9F41; CHRISTOPHER MA...,350.00,"$ 9,168.52",Dues,Jarod Maxson
3,3/9/2020,VENMO DES:CASHOUT ID:3205252458 INDN:TYLER MOT...,263.00,"$ 9,431.52",Dues,Tyler Mote
4,3/9/2020,VENMO DES:CASHOUT ID:3206423965 INDN:TYLER MOT...,250.00,"$ 9,681.52",Dues,Patrick Frazho
5,3/9/2020,VENMO DES:VERIFYBAN ID:3205250082 INDN:TYLER M...,0.16,"$ 9,681.68",NaN,NaN
6,3/9/2020,VENMO DES:VERIFYBAN ID:3205250079 INDN:TYLER M...,0.13,"$ 9,681.81",NaN,NaN
7,3/9/2020,VENMO DES:VERIFYBAN ID:3205250085 INDN:TYLER M...,-0.16,"$ 9,681.65",NaN,NaN
8,3/9/2020,VENMO DES:VERIFYBAN ID:3205250084 INDN:TYLER M...,-0.13,"$ 9,681.52",NaN,NaN
9,3/12/2020,VENMO DES:CASHOUT ID:3227545441 INDN:TYLER MOT...,250.00,"$ 9,931.52",Dues,Seth Hussey


In [4]:
df.columns = ['date', 'desc', 'amount', 'balance', 'category', 'payor/payee']

In [5]:
df.drop(columns=['payor/payee','category','balance','desc'], inplace=True)

In [6]:
df.head(10)

,date,amount
0,3/2/2020,"8,618.52"
1,3/2/2020,200.00
2,3/9/2020,350.00
3,3/9/2020,263.00
4,3/9/2020,250.00
5,3/9/2020,0.16
6,3/9/2020,0.13
7,3/9/2020,-0.16
8,3/9/2020,-0.13
9,3/12/2020,250.00


In [7]:
start = '8/1/2022'
end = '1/15/2023'

In [8]:
df['amount'] = df['amount'].str.replace(',','')
df['amount'] = pd.to_numeric(df['amount'])
df['date'] =  pd.to_datetime(df['date'])
df.dtypes

date      datetime64[ns]
amount           float64
dtype: object

In [9]:
fall22 = df[ (df['date']>start) & (df['date']<end)]

In [10]:
fall22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361 entries, 1148 to 1508
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    361 non-null    datetime64[ns]
 1   amount  361 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 8.5 KB


In [11]:
fall22 = fall22[fall22['amount']<0]
fall22.head(10)

,date,amount
1149,2022-08-02,-228.98
1150,2022-08-02,-1.00
1152,2022-08-03,-7.14
1163,2022-08-08,-51.14
1164,2022-08-09,-1134.44
1165,2022-08-09,-491.38
1168,2022-08-10,-256.44
1169,2022-08-10,-196.97
1171,2022-08-11,-866.69
1172,2022-08-11,-38.92


In [12]:
print(fall22['amount'].sum())

-69187.07


# looks way too high

In [13]:
fall22[fall22['amount']<-1000]

,date,amount
1164,2022-08-09,-1134.44
1194,2022-08-15,-7800.00
1267,2022-09-07,-1935.94
1295,2022-09-12,-2750.00
1306,2022-09-15,-1600.00
1321,2022-09-23,-1018.00
1365,2022-10-11,-1202.29
1383,2022-10-18,-1100.00
1384,2022-10-18,-5250.00
1393,2022-10-21,-4000.00


In [14]:
fall22.drop([1194,1267], inplace=True)

In [15]:
print(fall22['amount'].sum())

-59451.130000000005


## looks much better, now for calculating dues dates

In [29]:
total = fall22['amount'].sum()
fall22['totalspent'] = fall22['amount'].cumsum()
percentspent = 0.5 

In [28]:
fall22.loc[fall22['totalspent']>=total*percentspent, 'date'].iloc[-1]


Timestamp('2023-01-12 00:00:00')

Regular: 
8/15  - $250
9/05  - $185
9/30  - $185
10/25 - $130

PP:
8/15  - $150
8/25  - $100
9/05  - $90
9/18  - $90
9/30  - $90
10/14 - $90
10/25 - $75
11/10 - $65

OGP:
8/15  - $200
9/05  - $100
9/30  - $100
10/25 - $100

House Boy:
8/15  - $240
9/05  - $165
9/30  - $155
10/25 - $90

Brothers, above are the due dates for dues this coming semester. The dates have been picked in a way that due payments cover events before they happen, rather than reimbursing the bank account after the event. This means that from now on anyone who misses a payment will be barred from paid social events until they are caught up. Hopefully this will make us more secure financially because we won't have to pay for people who miss dues. 
i.e. if 10 people have outstanding dues, that's 10 people that we don't have to buy beer, liquor, spots on a bus etc. 
This doesn't necesarily include thirsty but depending on circumstance, like if you're not texting me at all or if it's like your third time being late, it might. 
We have big, ethnic, beautiful semester ahead of us fellas and these payments have to come in when they're needed. 
